# Amazon Augmented AI (Amazon A2I) integration with Amazon Rekognition [Example]

Visit https://github.com/aws-samples/amazon-a2i-sample-jupyter-notebooks for all A2I Sample Notebooks


1. [Introduction](#Introduction)
2. [Prerequisites](#Prerequisites)
    1. [Supported Regions](#Supported-Regions)
    2. [Workteam](#Workteam)
    3. [Notebook Permission](#Notebook-Permission)
3. [Client Setup](#Client-Setup)
4. [Sample Data](#Sample-Data)
    1. [Download sample images](#Download-sample-images)
    2. [Upload images to S3](#Upload-images-to-S3)
5. [Create Control Plane Resources](#Create-Control-Plane-Resources)
    1. [Create Human Task UI](#Create-Human-Task-UI)
    2. [Create Flow Definition](#Create-Flow-Definition)
6. [Scenario 1: When Activation Conditions are met, and a Human Loop is created](#Scenario-1-:-When-Activation-Conditions-are-met-,-and-HumanLoop-is-created)
    1. [Check Status of Human Loop](#Check-Status-of-Human-Loop)
    2. [Wait For Workers to Complete Task](#Wait-For-Workers-to-Complete-Task)
    3. [Check Status of Human Loop](#Check-Status-of-Human-Loop)
    4. [View Task Results](#View-Task-Results)
7. [Scenario 2: When Activation Conditions are not met, and a Human Loop is not created](#When-Activation-Conditions-are-not-met-,-and-HumanLoop-is-not-created)

## Introduction

Amazon Augmented AI (Amazon A2I) makes it easy to build the workflows required for human review of ML predictions. Amazon A2I brings human review to all developers, removing the undifferentiated heavy lifting associated with building human review systems or managing large numbers of human reviewers. 

Amazon A2I provides built-in human review workflows for common machine learning use cases, such as content moderation and text extraction from documents, which allows predictions from Amazon Rekognition and Amazon Textract to be reviewed easily. You can also create your own workflows for ML models built on Amazon SageMaker or any other tools. Using Amazon A2I, you can allow human reviewers to step in when a model is unable to make a high confidence prediction or to audit its predictions on an on-going basis. Learn more here: https://aws.amazon.com/augmented-ai/

In this tutorial, we will show how you can use Amazon A2I directly within your API calls to Amazon Rekognition's Detect Moderation Labels API. 

For more in depth instructions, visit https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-getting-started.html

To incorporate Amazon A2I into your human review workflows, you need three resources:

* A **worker task template** to create a worker UI. The worker UI displays your input data, such as documents or images, and instructions to workers. It also provides interactive tools that the worker uses to complete your tasks. For more information, see https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-instructions-overview.html

* A **human review workflow**, also referred to as a flow definition. You use the flow definition to configure your human workforce and provide information about how to accomplish the human review task. For built-in task types, you also use the flow definition to identify the conditions under which a review human loop is triggered. For example, with Amazon Rekognition can perform image content moderation using machine learning. You can use the flow definition to specify that an image will be sent to a human for content moderation review if Amazon Rekognition's confidence score output is low for your use case. You can create a flow definition in the Amazon Augmented AI console or with the Amazon A2I APIs. To learn more about both of these options, see https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-create-flow-definition.html

* A **human loop** to start your human review workflow. When you use one of the built-in task types, the corresponding AWS service creates and starts a human loop on your behalf when the conditions specified in your flow definition are met or for each object if no conditions were specified. When a human loop is triggered, human review tasks are sent to the workers as specified in the flow definition.

When using a custom task type, you start a human loop using the Amazon Augmented AI Runtime API. When you call StartHumanLoop in your custom application, a task is sent to human reviewers.

### Install Latest SDK

In [ ]:
# First, let's get the latest installations of our dependencies
!pip install --upgrade pip
!pip install boto3 --upgrade
!pip install -U botocore

## Setup
We need to set up the following data:
* `region` - Region to call A2I. Must be us-east-1 for Preview
* `bucket` - A S3 bucket accessible by the given role
    * Used to store the sample images & output results
    * Must be within the same region A2I is called from
* `role` - The IAM role used as part of StartHumanLoop. By default, this notebook will use the execution role
* `workteam` - Group of people to send the work to

### Supported Regions

For the Amazon A2I Preview, the **us-east-1** AWS region is supported. Please ensure that any accompanying resources are created in that region, including S3 buckets.

In [ ]:
# A2I is in us-east-1 for Preview
REGION = 'us-east-1'

#### Setup Bucket and Paths

In [ ]:
import boto3
import botocore

BUCKET = '<YOUR_BUCKET>'
OUTPUT_PATH = f's3://{BUCKET}/a2i-results'

### Notebook Permission

The AWS IAM Role used to execute the notebook needs to have the following permissions:

* RekognitionFullAccess
* SagemakerFullAccess
* S3 Read Access to the bucket listed below
* AmazonSageMakerMechanicalTurkAccess (if using MechanicalTurk as your Workforce)

In [ ]:
from sagemaker import get_execution_role

# Setting Role to the default SageMaker Execution Role
ROLE = get_execution_role()
display(ROLE)

Visit: https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-permissions-security.html to add the necessary permissions to your role

### Workteam or Workforce

A workforce is the group of workers that you have selected to label your dataset. You can choose either the Amazon Mechanical Turk workforce, a vendor-managed workforce, or you can create your own private workforce for human reviews. Whichever workforce type you choose, Amazon Augmented AI takes care of sending tasks to workers. 

When you use a private workforce, you also create work teams, a group of workers from your workforce that are assigned to Amazon Augmented AI human review tasks. You can have multiple work teams and can assign one or more work teams to each job.

To create your Workteam, visit the instructions here: https://docs.aws.amazon.com/sagemaker/latest/dg/sms-workforce-management.html

After you have created your workteam, replace YOUR_WORKTEAM_ARN below

In [ ]:
WORKTEAM_ARN= "<YOUR_WORKTEAM>"

## Client Setup

Here we are going to setup the clients. 

In [ ]:
import io
import json
import uuid
import time

# Amazon SageMaker client
sagemaker = boto3.client('sagemaker', REGION)

# Amazon Rekognition client
rekognition = boto3.client('rekognition', REGION)

# Amazon Augmented AI (A2I) Runtime client
a2i_runtime_client = boto3.client('sagemaker-a2i-runtime', REGION)

# S3 client
s3 = boto3.client('s3', REGION)

In [ ]:
import pprint

# Pretty print setup
pp = pprint.PrettyPrinter(indent=2)

# Function to pretty-print AWS SDK responses
def print_response(response):
    if 'ResponseMetadata' in response:
        del response['ResponseMetadata']
    pp.pprint(response)

## Sample Data

We'll be using sample images from Amazon Rekognition image moderation console.


### Download sample images

In [ ]:
# Download images
!wget 'https://dhei5unw3vrsx.cloudfront.net/images/family_picnic_resized.jpg' -O 'family_picnic_resized.jpg'
!wget 'https://dhei5unw3vrsx.cloudfront.net/images/yoga_swimwear_resized.jpg' -O 'yoga_swimwear_resized.jpg'


Yoga swimwear <img src="yoga_swimwear_resized.jpg" alt="Yoga Swimwear" width="400" />
Family picnic <img src="family_picnic_resized.jpg" alt="Family picnic" width="400" />


### Upload images to S3

Upload the sample images to your S3 bucket. They will be read by Amazon Rekognition and A2I later when human task is created.

In [ ]:
imageName1 = 'yoga_swimwear_resized.jpg'
imageName2 = 'family_picnic_resized.jpg'

In [ ]:
s3.upload_file(imageName1, BUCKET, imageName1)
s3.upload_file(imageName2, BUCKET, imageName2)

You can now check the S3 bucket BUCKET that it contains the images at the specified key paths.

## Create Control Plane Resources

Here we'll be constructing the following control plane resources: Human Task UI and Flow Definition, using the SageMaker CreateTaskUI and CreateFlowDefinition APIs, respectively.

These resources can be created once and used to drive any subsequent A2I human loops.

### Create Human Task UI

Create a human task UI resource, giving a UI template in liquid html. This template will be rendered to the workers whenever a human loop is created.

Amazon A2I provides pre-built templates for Amazon Rekognition and Amazon Tetract. Additionally, over 70  different samples UIs for different use cases are here: https://github.com/aws-samples/amazon-a2i-sample-task-uis

In [ ]:
template = r"""
<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>
{% capture s3_arn %}http://s3.amazonaws.com/{{ task.input.aiServiceRequest.image.s3Object.bucket }}/{{ task.input.aiServiceRequest.image.s3Object.name }}{% endcapture %}

<crowd-form>
  <crowd-rekognition-detect-moderation-labels
    categories='[
      {% for label in task.input.selectedAiServiceResponse.moderationLabels %}
        {
          name: "{{ label.name }}",
          parentName: "{{ label.parentName }}",
        },
      {% endfor %}
    ]'
    src="{{ s3_arn | grant_read_access }}"
    header="Please select all categories that apply"
  >
    <short-instructions header="Instructions">
      <style>
        .instructions {
          white-space: pre-wrap;
        }
      </style>
      <p class='instructions'>Review the image and choose all applicable categories.
        If no categories apply, choose None.

        <b>Nudity</b>
        Visuals depicting nude male or female person or persons

        <b>Graphic Male Nudity</b>
        Visuals depicting full frontal male nudity, often close ups

        <b>Graphic Female Nudity</b>
        Visuals depicting full frontal female nudity, often close ups

        <b>Sexual Activity</b>
        Visuals depicting various types of explicit sexual activities and pornography

        <b>Illustrated Nudity or Sexual Activity</b>
        Visuals depicting animated or drawn sexual activity, nudity or pornography

        <b>Adult Toys</b>
        Visuals depicting adult toys, often in a marketing context

        <b>Female Swimwear or Underwear</b>
        Visuals depicting female person wearing only swimwear or underwear

        <b>Male Swimwear Or Underwear</b>
        Visuals depicting male person wearing only swimwear or underwear

        <b>Partial Nudity</b>
        Visuals depicting covered up nudity, for example using hands or pose

        <b>Revealing Clothes</b>
        Visuals depicting revealing clothes and poses, such as deep cut dresses

        <b>Graphic Violence or Gore</b>
        Visuals depicting prominent blood or bloody injuries

        <b>Physical Violence</b>
        Visuals depicting violent physical assault, such as kicking or punching

        <b>Weapon Violence</b>
        Visuals depicting violence using weapons like firearms or blades, such as shooting

        <b>Weapons</b>
        Visuals depicting weapons like firearms and blades

        <b>Self Injury</b>
        Visuals depicting self-inflicted cutting on the body, typically in distinctive patterns using sharp objects

        <b>Emaciated Bodies</b>
        Visuals depicting extremely malnourished human bodies

        <b>Corpses</b>
        Visuals depicting human dead bodies

        <b>Hanging</b>
        Visuals depicting death by hanging</p>
    </short-instructions>

    <full-instructions header="Instructions"></full-instructions>
  </crowd-rekognition-detect-moderation-labels>
</crowd-form>
"""

def create_task_ui():
    '''
    Creates a Human Task UI resource.

    Returns:
    struct: HumanTaskUiArn
    '''
    response = sagemaker.create_human_task_ui(
        HumanTaskUiName=taskUIName,
        UiTemplate={'Content': template})
    return response

In [ ]:
# Task UI name - this value is unique per account and region. You can also provide your own value here.
taskUIName = 'ui-rekognition-demo-' + str(uuid.uuid4())

humanTaskUiResponse = create_task_ui()
humanTaskUiArn = humanTaskUiResponse['HumanTaskUiArn']
print(humanTaskUiArn)

### Create Flow Definition

Now we can create a Flow Definition. The Flow Definition encapsulates the following high-level concepts:

* The AWS managed request source (such as AWS Rekognition content moderation **AWS/Rekognition/Image/ContentModeration/V3**)
* Business Conditions against which ML inference is evaluated and a human loop is created if conditions are met
* Workteam to review the human tasks, number of workers per task and other details
* Task UI template
* Output S3 location for the human task results

Flow Definition is associated with a particular AWS managed request source, which affects the structure of the human loop activation conditions and format of the inference input and result.

The human loop activation conditions used in this demo are tailored towards Amazon Rekognition content moderation - they are based on the confidence thresholds for particular moderation labels.

Activation conditions can be expressed using logical operators *And*, *Or* etc.

In [ ]:
def create_flow_definition():
    '''
    Creates a Flow Definition resource

    Returns:
    struct: FlowDefinitionArn
    '''
    humanLoopActivationConditions = json.dumps(
        {
            "Conditions": [
                {
                  "Or": [
                    {
                        "ConditionType": "ModerationLabelConfidenceCheck",
                        "ConditionParameters": {
                            "ModerationLabelName": "Suggestive",
                            "ConfidenceLessThan": 98
                        }
                    },
                    {
                        "ConditionType": "ModerationLabelConfidenceCheck",
                        "ConditionParameters": {
                            "ModerationLabelName": "Female Swimwear Or Underwear",
                            "ConfidenceGreaterThan": 98
                        }
                    }
                  ]
               }
            ]
        }
    )

    response = sagemaker.create_flow_definition(
            FlowDefinitionName= flowDefinitionName,
            RoleArn= ROLE,
            HumanLoopConfig= {
                "WorkteamArn": WORKTEAM_ARN,
                "HumanTaskUiArn": humanTaskUiArn,
                "TaskCount": 1,
                "TaskDescription": "Demo A2I moderation sample task description",
                "TaskTitle": "Demo A2I moderation sample task"
            },
            HumanLoopRequestSource= {
                "AwsManagedHumanLoopRequestSource": "AWS/Rekognition/DetectModerationLabels/Image/V3"
            },
            HumanLoopActivationConfig={
                "HumanLoopActivationConditionsConfig": {
                    "HumanLoopActivationConditions": humanLoopActivationConditions
                }
            },
            OutputConfig={
                "S3OutputPath" : OUTPUT_PATH
            }
        )
    print_response(response)
    return response['FlowDefinitionArn'] 

In [ ]:
# Flow definition name - this value is unique per account and region. You can also provide your own value here.
flowDefinitionName = 'fd-rekognition-demo-' + str(uuid.uuid4())

# Create a flow definition. We'll be using the flow definition arn for starting human loops.
flow_definition_arn = create_flow_definition()

In [ ]:
def describe_flow_definition(name):
    '''
    Describes Flow Definition

    Returns:
    struct: response from DescribeFlowDefinition API invocation
    '''
    return sagemaker.describe_flow_definition(
        FlowDefinitionName=name)

# Describe flow definition - status should be active
for x in range(60):
    describeFlowDefinitionResponse = describe_flow_definition(flowDefinitionName)
    print(describeFlowDefinitionResponse['FlowDefinitionStatus'])
    if (describeFlowDefinitionResponse['FlowDefinitionStatus'] == 'Active'):
        print("Flow Definition is active")
        break
    time.sleep(2)

### Detect Moderation Labels with AWS Rekognition

Let's call Amazon Rekognition to detect moderation labels on the sample images stored in S3 based on the steps above.

In [ ]:
uniqueId = str(uuid.uuid4())

def detect_moderation_labels(img_name):
    response = rekognition.detect_moderation_labels(Image={'S3Object': {'Bucket': BUCKET, 'Name': img_name}}, 
                                                    HumanLoopConfig={'HumanLoopName': uniqueId + '-1', 'FlowDefinitionArn': flow_definition_arn})
    return response

## Scenario 1: When Activation Conditions are met, a Human Loop is created

When an image passed to Rekognition matches the conditions in FlowDefinition, a HumanLoopArn will be present in the response to detect_moderation_labels. 

In [ ]:
moderationResponse1 = detect_moderation_labels(imageName1)

In [ ]:
if 'HumanLoopArn' in moderationResponse1['HumanLoopActivationOutput']:
    # A human loop has been started!
    print(f'A human loop has been started: {moderationResponse1["HumanLoopActivationOutput"]["HumanLoopArn"]}')

### Check Status of Human Loop

In [ ]:
all_human_loops_in_workflow = a2i_runtime_client.list_human_loops(FlowDefinitionArn=flow_definition_arn)['HumanLoopSummaries']

for human_loop in all_human_loops_in_workflow:
    print(f'\nHuman Loop Name: {human_loop["HumanLoopName"]}')
    print(f'Human Loop Status: {human_loop["HumanLoopStatus"]} \n')
    print('\n')
    if human_loop['HumanLoopStatus'] == 'Completed':
        completed_loops.append(human_loop['HumanLoopName'])


### Wait For Workers to Complete Task

In [ ]:
workteamName = WORKTEAM_ARN[WORKTEAM_ARN.rfind('/') + 1:]
print("Navigate to the private worker portal and do the tasks. Make sure you've invited yourself to your workteam!")
print('https://' + sagemaker.describe_workteam(WorkteamName=workteamName)['Workteam']['SubDomain'])

### Check Status of Human Loop

In [ ]:
all_human_loops_in_workflow = a2i_runtime_client.list_human_loops(FlowDefinitionArn=flow_definition_arn)['HumanLoopSummaries']

completed_loops = []
for human_loop in all_human_loops_in_workflow:
    print(f'\nHuman Loop Name: {human_loop["HumanLoopName"]}')
    print(f'Human Loop Status: {human_loop["HumanLoopStatus"]} \n')
    print('\n')
    if human_loop['HumanLoopStatus'] == 'Completed':
        completed_loops.append(human_loop['HumanLoopName'])


### View Task Results  

Once work is completed, Amazon A2I stores results in your S3 bucket and sends a Cloudwatch event. Your results should be available in the S3 OUTPUT_PATH when all work is completed.

In [ ]:
import re
import pprint
pp = pprint.PrettyPrinter(indent=2)

def retrieve_a2i_results_from_output_s3_uri(bucket, a2i_s3_output_uri):
    '''
    Gets the json file published by A2I and returns a deserialized object
    '''
    splitted_string = re.split('s3://' +  bucket + '/', a2i_s3_output_uri)
    output_bucket_key = splitted_string[1]

    response = s3.get_object(Bucket=bucket, Key=output_bucket_key)
    content = response["Body"].read()
    return json.loads(content)
    

for human_loop_name in completed_loops:

    describe_human_loop_response = a2i_runtime_client.describe_human_loop(
        HumanLoopName=human_loop_name
    )
    
    print(f'\nHuman Loop Name: {describe_human_loop_response["HumanLoopName"]}')
    print(f'Human Loop Status: {describe_human_loop_response["HumanLoopStatus"]}')
    print(f'Human Loop Output Location: : {describe_human_loop_response["HumanLoopOutput"]["OutputS3Uri"]} \n')
    
    # Uncomment below line to print out a2i human answers
    # pp.pprint(retrieve_a2i_results_from_output_s3_uri(BUCKET, describe_human_loop_response['HumanLoopOutput']['OutputS3Uri']))


## Scenario 2: When Activation Conditions are not met, Human Loop is not created

Image passed to Rekognition does not match the conditions in FlowDefinition so HumanLoopArn will not be present in the response.

In [ ]:
moderationResponse2 = detect_moderation_labels(imageName2)

In [ ]:
if 'HumanLoopArn' in moderationResponse2['HumanLoopActivationOutput']:
    # A human loop has been started!
    print(f'A human loop has been started: {moderationResponse1["HumanLoopActivationOutput"]["HumanLoopArn"]}')
else:
    print('No human loop started as none of the HumanLoopActivationConditions were met.')

Since no **HumanLoopArn** is present, this implies that no HumanLoopActivationConditions were met and no Human Loop was created.

## The End!